In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import json

# 네이버 영화 code 구하기

In [ ]:
# https://movie.naver.com/movie/search/result.nhn?query=기생충&section=all&ie=utf8

In [ ]:
def searchMovieName(movieName):
    MOVIE_SERCH_URL = "https://movie.naver.com/movie/search/result.nhn?"
    # query=기생충&section=all&ie=utf8
    # movieName = "기생충"
    params = {
        "query" : movieName,
        "section" : "all",
        "ie" : "utf8"
    }

    resp = requests.get(MOVIE_SERCH_URL, params = params)
    soup = BeautifulSoup(resp.content, "html.parser")

    movieCode = soup.find("p", class_ = "result_thumb").a["href"][-6:]

    return movieCode

In [ ]:
resp = requests.get(MOVIE_SERCH_URL, params = params)
soup = BeautifulSoup(resp.content, "html.parser")

In [ ]:
movieCode = soup.find("p", class_ = "result_thumb").a["href"][-6:]


# 네이버 영화 리뷰 Crawling

In [ ]:
# review basic
def movieReviewCrawling(movieCode):
    categoryDic = {
        "review" : "review",
        "reviewread" : "reviewread",
        "end" : ".nhn?"
    }

    # code=136900
    MOVIE_URL = "https://movie.naver.com"
    TAG = "/movie/bi/mi/"
    REVIEW_URL = MOVIE_URL + TAG + categoryDic['review'] + categoryDic['end']
    # movieCode = "136900"

    params = {
        "code" : movieCode
    }

    resp = requests.get(REVIEW_URL, params = params)
    
    soup = BeautifulSoup(resp.content, "html.parser")
    ulContents = soup.find("ul", class_ = "rvw_list_area")
    return doCrawling(ulContents)


In [ ]:
soup = BeautifulSoup(resp.content, "html.parser")
ulContents = soup.find("ul", class_ = "rvw_list_area")
pagingContents = soup.find("div", class_ = "paging")#.find_next_sibling

In [ ]:
reviewPageList = []
for div in pagingContents:
    if(div == "\n"):
        continue
    for i in div:
        if(i == "\n"):
            continue
        reviewPageList.append(i)

reviewPageList.pop()


In [ ]:
reviewPageList

In [ ]:
for i in reviewPageList:
    print(MOVIE_URL+i["href"])

In [ ]:
def doCrawling(ulContents):
    reviewList = []
    # print(list(ulContents))
    for li in ulContents:
        reviewDic = {}

        if(li == "\n"):
            continue

        liSpan = li.span
        nid = str(liSpan).split("showReviewListByNid")[1][2:9]
        reviewDic["title"] = li.find("a").text.strip()
        reviewDic["date"] = liSpan.em.text.strip()
        reviewDic["recommend"] = liSpan.em.find_next_sibling().text[3:].strip()
        reviewDic["previewText"] = liSpan.find_next_sibling().a.text.strip()
        reviewDic["nid"] = nid

        # detail ----------------------------------------------------

        DETAIL_REVIEW_URL = MOVIE_URL + TAG + categoryDic['reviewread'] + categoryDic['end']

        detailParams = {
            "nid" : nid,
            "code" : movieCode
        }

        detailResp = requests.get(DETAIL_REVIEW_URL, params = detailParams)
        detailSoup = BeautifulSoup(detailResp.content, "html.parser")
        detailDivContents = detailSoup.find_all("div", class_ = "user_tx_area")
        tempStr = ""
#         print(detailDivContents)
        for i in detailDivContents:
            temp = i.find("span")
            if type(temp) != int and temp != None and temp.text != "\n":
                tempStr += temp.text

        # detail ----------------------------------------------------

        reviewDic["fullText"] = tempStr


        reviewList.append(reviewDic)
    return reviewList

In [ ]:
# https://movie.naver.com/movie/bi/mi/review.nhn?code=136900
# https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4007591&code=136900
# https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4699039&code=136900&order=#tab
# reviewList

# Main method

In [ ]:
movieName = input("영화 제목 입력: ")
movieCode = searchMovieName(movieName)
reviewList = movieReviewCrawling(movieCode)

pprint(reviewList)
# with open(f"{movieName}.json", 'w', encoding='utf-8') as file :
#         json.dump(reviewList, file, ensure_ascii=False, indent='\t')